# Reading persian data

In [105]:
!pip install https://github.com/sobhe/hazm/archive/master.zip --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     \ 36.3 MB 29.5 MB/s 0:00:02
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 29.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 KB 29.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for hazm: filename=hazm-0.7.1-py3-none-any.whl size=480670 sha256=c22ba711c997be5b044cefddc8861925e83cdde0bc878a18217304ec30fef6c7
  Stored in directory: /tmp/pip-ephem-wheel-cache-zgq3mejf/wheels/d0/ff/8c/96985b925e870d68b96fecf834358f9742818c8b41e1abfff9
  Created wheel for nltk: filename=nltk-3.4-py3-none-any.whl size=1436394 sha256=2be36e5d0bcfe9560dbc387e2ac4608dea5275cb146ee74a3b9f31d93ce85ec9
  Stored in directory: /root/.cache/pip/wheels/db/96/da/0a26fbd3f96b179cc14b813434a0c324a08c0684afdd524c73
  Created wheel for libwapiti: filename=libwapiti-0.2.1-cp3

In [1]:
from hazm import Normalizer

In [1]:
from nltk.util import pad_sequence
from nltk.util import bigrams
from nltk.util import ngrams
from nltk.util import everygrams
from nltk.lm import Laplace
from nltk.lm import MLE
from nltk.lm.preprocessing import pad_both_ends
from nltk.lm.preprocessing import flatten
import re
import random
from nltk import word_tokenize, sent_tokenize 
from nltk.lm.preprocessing import padded_everygram_pipeline
import os
import io #codecs
import nltk
from itertools import product
import math
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# A) Cleaning the data
 In the given Persian text, there are some random non-Persian characters which should be removed. Therefore, we remove all non-Persian and extra characters (except for punctuation marks). Then, using the implemented normalizer in the Hazm library, we correct the words in the file, which includes correcting spaces and words. For example, in the provided data file version, all "ی" letters are written as "ي," which is incorrect. (If this correction was not made, we would encounter a problem in future sections of this question  "هری" where this word did not exist in the entire book). Furthermore, after segmenting the sentences using the "<s>" and "</s>" characters, we identify the beginning and end of each sentence.

In [3]:
hp_fa=open("hp_fa.txt", "r")

In [4]:
if os.path.isfile('hp_fa.txt'):
    with io.open('hp_fa.txt', encoding='utf8') as fin:
        text = fin.read()

In [5]:
def persian_only(s):
    return " ".join(re.findall(r"[\u0600-\u06FF|\u003a|\u003B|\u0021|\u003f|(|)|.|!|؟|?|\u2324|\u000a]+", s))

In [6]:
text=persian_only(text)

In [7]:
text

'آقا و خانم دورسلي ساکن خانه شماره چهار خيابان پريوت درايو بودند. خانواده آنها بسيار معمولي و عادي بود و آن ها از اين بابت بسيار راضي و خوشنود بودند. اين خانواده به هيچ وجه با امور مرموز و اسرار آميز سروکار نداشتند زيرا سحر و جادو را امر مهمل و بيهوده اي مي پنداشتند و علاقه اي به اين گونه مسائل نداشت ن\nآقاي دورسلي مدير شرکت دريل سازي گرونينگز ، مردي درشت اندام و قوي هيکل بود با گردني بسيار کوتاه که سبيل بلندي نيز داشت . همسر او، خانم دورسلي زني لاغر اندام بود با موهاي بور و گردني کشيده و بلند. بلندي گردنش بسيار برايش مفيد بود زيرا بيش تر وقتش را صرف سرک کشيدن به خانه ي همسايه ها مي کرد. آنها پسري داشتند به نام دادلي که به عقيده خودشان لنگه نداشت . \nآنها خانواده ي مرفهي بودند و هيچ کم و کسري نداشتند اما در اين خانواده رازي وجود داشت که نبايد برملا مي شد. آن ها هميشه در هول و هراس بودند که مبادا روزي کسي به رازشان پي ببرد. حتي تصور اين که کسي از خانواده ي پاتر چيزي بداند برايشان غيرقابل تحمل بود. خانم پاتر خواهر خانم دورسلي بود اما سال ها بود که يکديگر را نديده بودند. در واقع خانم دورس

In [8]:
normalizer = Normalizer()
text=str(normalizer.normalize(text))

In [9]:
f = open("Cleaned_hp_fa.txt", "a")
f.write(text)
f.close()

In [10]:
if os.path.isfile('Cleaned_hp_fa.txt'):
    with io.open('Cleaned_hp_fa.txt', encoding='utf8') as fin:
        text = fin.read()

In [11]:
tokenized_text = [list(word_tokenize(sent)) 
                  for sent in sent_tokenize(text)]

#B) N=2


# C) Laplace

In [21]:
n = 2
train_data, padded_sents = padded_everygram_pipeline(n, tokenized_text)

In [22]:
bigram_model = MLE(1)

In [23]:
bigram_model.fit(train_data, padded_sents)

# D) Genrate

In [14]:
def sent_genrator(model, start=12,finish=24):
  sent=model.generate(random.randint(start, finish))
  sentence=""
  for word in sent:
      if word !="<s>" and word!="</s>":
        sentence=sentence+ " "+ word
  return sentence
def sentence_genrator(model, start=12,finish=24,number_of_senteces=5):
  sentences=[]
  for _ in range(number_of_senteces):
    sent=""
    while (len(sent.split())<12):
      sent=sent_genrator(model, start=12,finish=24)
    sentences.append(sent)
  return sentences

In [25]:
sentences=sentence_genrator(bigram_model,12,24,5)
for sent in sentences:
  print(sent)

 بود سراغ جیم ! انتقام بگیرند و رون گفت : بله، بر روی سگ سه بار به انتهای گذرگاه به جارویش شود
 شد که زیرگوش چپش یک واقعه عجیب و من می‌دونم اون بالاخره چه طوری رفتار
 شیشه‌ی پنجره سرک می‌کشید فشار باز همه چیز باشه که چنین چیزی سابقه نداشته ! این آخرین لحظه وود صدایش غم
 از مدت زیادی از ماه‌ها آن جا پرید : هیچی، داشتم از آن‌ها
 … این قدر خیس کرد . کلبه برخورد بهتری به آن‌ها از خوابگاه خارج


#E) N=3 , N=5


## N=3 

In [12]:
n = 3
train_data, padded_sents = padded_everygram_pipeline(n, tokenized_text)
threegram_model = Laplace(1)
threegram_model.fit(train_data, padded_sents)

In [15]:
sentences=sentence_genrator(threegram_model,12,24,5)
for sent in sentences:
  print(sent)

 قرار بود گريفندور و سرسراي بزرگ آمدند توجه همه را به آن‌ها
 تنها چيزي که عمو ورنون که خشمگين و برافروخته بود که صداي خرناس دادلي ديگر به تن داشت و با صورت
 کنارشان ميگذشت با خشم گفت : نمیدونین با این حال میتونی یه سؤال بود
 تا سرش باشه ؟ اما هري هيچ وقت اسنيپ رو مدتي سرگرم کنم و بهش دروغ بگم .
 او سر کلاس بيوقفه حرف ميزد و زير لب گفت که هیچ‌یک از این گوشته


## N=5


In [16]:
n = 5
train_data, padded_sents = padded_everygram_pipeline(n, tokenized_text)
fivegram_model = Laplace(1)
fivegram_model.fit(train_data, padded_sents)

In [17]:
sentences=sentence_genrator(fivegram_model,12,24,5)
for sent in sentences:
  print(sent)

 وود با دقت گوی زرین را در صندوق گذاشت و گفت : فقط سی و شش تا کادو؟
 گروه بندي بشن ، درسته ؟ چند دانش آموز با حرکت سر حرف او را
 به نظر می‌رسید هری را شناخته باشد گفت : بله ، خيلي پردردسر بود .
 ) ، يکي اتاق دادلي بود و در آخرين اتاق نيز اسباب بازيهاي اضافي و شکسته
 در سالن عمومي گريفندور و سرسراي بزرگ ، آتش مطبوعي در بخاري هاي ديواري زبانه مي کشيد و همه جا


# F)
The 5-gram model has better results than the other two models. It is clear that this model pays more attention to the number of previous words, and as a result, it will have a structure similar to a real sentence.


# Without using library 

In [12]:

SOS = "<s> "
EOS = "</s>"
UNK = "<UNK>"

def add_sentence_tokens(sentences, n):
    sos = SOS * (n-1) if n > 1 else SOS
    return ['{}{} {}'.format(sos, s, EOS) for s in sentences]

def replace_singletons(tokens):
    vocab = nltk.FreqDist(tokens)
    return [token if vocab[token] > 1 else UNK for token in tokens]

def preprocess(sentences, n):

    sentences = add_sentence_tokens(sentences, n)
    tokens = ' '.join(sentences).split(' ')
    tokens = replace_singletons(tokens)
    return tokens

In [13]:

class LanguageModel(object):


    def __init__(self, train_data, n, laplace=1):
        self.n = n
        self.laplace = laplace
        self.tokens = preprocess(train_data, n)
        self.vocab  = nltk.FreqDist(self.tokens)
        self.model  = self._create_model()
        self.masks  = list(reversed(list(product((0,1), repeat=n))))

    def _smooth(self):

        vocab_size = len(self.vocab)

        n_grams = nltk.ngrams(self.tokens, self.n)
        n_vocab = nltk.FreqDist(n_grams)

        m_grams = nltk.ngrams(self.tokens, self.n-1)
        m_vocab = nltk.FreqDist(m_grams)

        def smoothed_count(n_gram, n_count):
            m_gram = n_gram[:-1]
            m_count = m_vocab[m_gram]
            return (n_count + self.laplace) / (m_count + self.laplace * vocab_size)

        return { n_gram: smoothed_count(n_gram, count) for n_gram, count in n_vocab.items() }

    def _create_model(self):

        if self.n == 1:
            num_tokens = len(self.tokens)
            return { (unigram,): count / num_tokens for unigram, count in self.vocab.items() }
        else:
            return self._smooth()

    def _convert_oov(self, ngram):

        mask = lambda ngram, bitmask: tuple((token if flag == 1 else "<UNK>" for token,flag in zip(ngram, bitmask)))

        ngram = (ngram,) if type(ngram) is str else ngram
        for possible_known in [mask(ngram, bitmask) for bitmask in self.masks]:
            if possible_known in self.model:
                return possible_known

    def perplexity(self, test_data):
        test_tokens = preprocess(test_data, self.n)
        test_ngrams = nltk.ngrams(test_tokens, self.n)
        N = len(test_tokens)

        known_ngrams  = (self._convert_oov(ngram) for ngram in test_ngrams)
        probabilities = [self.model[ngram] for ngram in known_ngrams]

        return math.exp((-1/N) * sum(map(math.log, probabilities)))

    def _best_candidate(self, prev, i, without=[]):
        blacklist  = ["<UNK>"] + without
        candidates = ((ngram[-1],prob) for ngram,prob in self.model.items() if ngram[:-1]==prev)
        candidates = filter(lambda candidate: candidate[0] not in blacklist, candidates)
        candidates = sorted(candidates, key=lambda candidate: candidate[1], reverse=True)
        if len(candidates) == 0:
            return ("</s>", 1)
        else:
            return candidates[0 if prev != () and prev[-1] != "<s>" else i]
     
    def generate_sentences(self, num, min_len=12, max_len=24):
        for i in range(num):
            sent, prob = ["<s>"] * max(1, self.n-1), 1
            while sent[-1] != "</s>":
                prev = () if self.n == 1 else tuple(sent[-(self.n-1):])
                blacklist = sent + (["</s>"] if len(sent) < min_len else [])
                next_token, next_prob = self._best_candidate(prev, i, without=blacklist)
                sent.append(next_token)
                prob *= next_prob
                
                if len(sent) >= max_len:
                    sent.append("</s>")

            yield ' '.join(sent) 
       

In [20]:
class LanguageModel(object):
    def __init__(self, train_data, n, laplace=1):
        self.n = n
        self.laplace = laplace
        self.tokens = preprocess(train_data, n)
        self.vocab  = nltk.FreqDist(self.tokens)
        self.model  = self._create_model()
        self.masks  = list(reversed(list(product((0,1), repeat=n))))

    def _smooth(self):

        vocab_size = len(self.vocab)

        n_grams = nltk.ngrams(self.tokens, self.n)
        n_vocab = nltk.FreqDist(n_grams)

        m_grams = nltk.ngrams(self.tokens, self.n-1)
        m_vocab = nltk.FreqDist(m_grams)

        def smoothed_count(n_gram, n_count):
            m_gram = n_gram[:-1]
            m_count = m_vocab[m_gram]
            return (n_count + self.laplace) / (m_count + self.laplace * vocab_size)

        return { n_gram: smoothed_count(n_gram, count) for n_gram, count in n_vocab.items() }

    def _create_model(self):
        if self.n == 1:
            num_tokens = len(self.tokens)
            return { (unigram,): count / num_tokens for unigram, count in self.vocab.items() }
        else:
            return self._smooth()

    def _convert_oov(self, ngram):
        mask = lambda ngram, bitmask: tuple((token if flag == 1 else "<UNK>" for token,flag in zip(ngram, bitmask)))

        ngram = (ngram,) if type(ngram) is str else ngram
        for possible_known in [mask(ngram, bitmask) for bitmask in self.masks]:
            if possible_known in self.model:
                return possible_known

    def perplexity(self, test_data):
        test_tokens = preprocess(test_data, self.n)
        test_ngrams = nltk.ngrams(test_tokens, self.n)
        N = len(test_tokens)

        known_ngrams  = (self._convert_oov(ngram) for ngram in test_ngrams)
        probabilities = [self.model[ngram] for ngram in known_ngrams]

        return math.exp((-1/N) * sum(map(math.log, probabilities)))

    def _best_candidate(self, prev, i, without=[]):
        blacklist  = ["<UNK>"] + without
        candidates = ((ngram[-1],prob) for ngram,prob in self.model.items() if ngram[:-1]==prev)
        candidates = filter(lambda candidate: candidate[0] not in blacklist, candidates)
        candidates = sorted(candidates, key=lambda candidate: candidate[1], reverse=True)
        if len(candidates) == 0:
            return ("</s>", 1)
        else:
            return candidates[0 if prev != () and prev[-1] != "<s>" else i]
     
    def generate_sentences(self, num, min_len=12, max_len=24):
        for i in range(num):
            sent, prob = ["<s>"] * max(1, self.n-1), 1
            while sent[-1] != "</s>":
                prev = () if self.n == 1 else tuple(sent[-(self.n-1):])
                blacklist = sent + (["</s>"] if len(sent) < min_len else [])
                next_token, next_prob = self._best_candidate(prev, i, without=blacklist)
                sent.append(next_token)
                prob *= next_prob
                
                if len(sent) >= max_len:
                    sent.append("</s>")

            yield ' '.join(sent)

In [15]:
with open("/content/Cleaned_hp_fa.txt", 'r') as f:
      train = [l.strip() for l in f.readlines()]
#train=persian_only(train)    
lm2 = LanguageModel(train, 2, laplace=1)
print("Vocabulary size: {}".format(len(lm2.vocab)))

print("Generating sentences...")
for sentence in lm2.generate_sentences(12):
    print("{} )".format(sentence))


Vocabulary size: 5090
Generating sentences...
<s>  </s> )
<s> هری گفت: ای کاش نامه‌اش را به نظر می‌رسید که از این جا دور و با صدای بلند شد … </s> )
<s> رون گفت: ای کاش نامه‌اش را به نظر می‌رسید که از این جا دور و با صدای بلند شد … </s> )
<s> هاگرید گفت: ای کاش نامه‌اش را به نظر می‌رسید که از این جا دور و با صدای بلند شد … </s> )
<s> هرمیون گفت: ای کاش نامه‌اش را به نظر می‌رسید که از این جا دور و با صدای بلند شد … </s> )
<s> در آن را به نظر می‌رسید که از این جا دور و گفت: </s> )
<s> این جا که از آن را به نظر می‌رسید و گفت: </s> )
<s> پس از این جا که به نظر می‌رسید و گفت: ای کاش نامه‌اش را در آن گاه صدای بلند شد با هم توی </s> )
<s> من که از این جا دور و گفت: ای کاش نامه‌اش را به نظر می‌رسید آن‌ها در آن گاه صدای بلند شد با </s> )
<s> اما هری گفت: ای کاش نامه‌اش را به نظر می‌رسید که از این جا دور و با صدای بلند شد … </s> )
<s> پروفسور مک گونگال گفت: ای کاش نامه‌اش را به نظر می‌رسید که از این جا دور و با صدای بلند شد … </s> </s> )
<s> به نظر می‌رسید که از این جا دور و گفت: ای کاش نامه‌اش ر

In [16]:
lm3 = LanguageModel(train, 3, laplace=1)
print("Vocabulary size: {}".format(len(lm3.vocab)))

print("Generating sentences...")
for sentence in lm3.generate_sentences(12):
    print("{} )".format(sentence))


Vocabulary size: 5090
Generating sentences...
<s> <s>  </s> )
<s> <s> هری که از آن جا دور شد. دادلی با اوقات تلخی گفت: </s> )
<s> <s> رون گفت: </s> )
<s> <s> هاگرید با حالتی عصبی انگشت هایش را به سمت هری رفت. هنگامی‌که </s> )
<s> <s> هرمیون گفت: </s> )
<s> <s> در همان لحظه صدای فریاد وحشت‌زده‌ای از پشت سر هم چوبدستی‌ها را امتحان می‌کرد اما هری و رون به او نگاه کرد </s> )
<s> <s> این جا رو نگاه کنین! نیمبوس دو </s> )
<s> <s> پس از مدتی شروع به صحبت کرد و گفت: وینگاردیوم له ویوسا! بلافاصله پر </s> )
<s> <s> من به او نگاه کرد و گفت: وینگاردیوم له ویوسا! بلافاصله پر از تخم بیرون اومد باید خون جوجه رو با همچین </s> )
<s> <s> اما هری و رون به او نگاه کرد که در آن لحظه تنها چیزی بود </s> )
<s> <s> پروفسور مک گونگال گفت: </s> )
<s> <s> به نظر می‌رسید که اصلا موهایش را کوتاه کرد و گفت: </s> )


In [17]:
lm5 = LanguageModel(train, 5, laplace=1)
print("Vocabulary size: {}".format(len(lm5.vocab)))

print("Generating sentences...")
for sentence in lm5.generate_sentences(12):
    print("{} )".format(sentence))

Vocabulary size: 5090
Generating sentences...
<s> <s> <s> <s>  </s> )
<s> <s> <s> <s> هری که از قطع شدن گفتگویش با آن پسر اصلا ناراحت نبود </s> )
<s> <s> <s> <s> رون گفت: </s> )
<s> <s> <s> <s> هاگرید با حالتی صمیمانه و </s> )
<s> <s> <s> <s> هرمیون گفت: </s> )
<s> <s> <s> <s> در همان لحظه دامبلدور از روی دیوار کوتاه </s> )
<s> <s> <s> <s> این جا رو نگاه کنین! نیمبوس دو </s> )
<s> <s> <s> <s> پس از رفتن فرد و جرج بلافاصله سروکله‌ی کسی پیدا شد که چندان دوست‌داشتنی نبود. مالفوی در میان کراب </s> </s> )
<s> <s> <s> <s> من به او اعتماد کامل دارم. </s> )
<s> <s> <s> <s> اما هری متوجه شد که این بار </s> )
<s> <s> <s> <s> پروفسور مک گونگال با غضب بینی‌اش را بالا کشید و ساعت </s> )
<s> <s> <s> <s> به نظر می‌رسید که اصلا موهایش را کوتاه نکرده است. خاله پتونیا از کوره در رفت و با </s> )


# G) 
The sentence of prayer is the pillar of religion will be more complicated. Because the word prayer or even religion does not exist in the text of the Harry Potter book, and as a result, the probability of this word occurring will be close to zero, and the complexity of this sentence will be very large.

In [18]:
sent1="هری به هاگوارتز برگشت."
models=[lm2,lm3,lm5]
model_names=["2-gram","3-gram","5-gram"]
for i in range(3):
    print("The perplexity of {} for the {} model is equal to:".format(sent1,model_names[i])+ str(models[i].perplexity(sent1)))

The perplexity of هری به هاگوارتز برگشت. for the 2-gram model is equal to:19.45993316778004
The perplexity of هری به هاگوارتز برگشت. for the 3-gram model is equal to:32.24617727326568
The perplexity of هری به هاگوارتز برگشت. for the 5-gram model is equal to:58.77820527871448


In [19]:
sent2="نماز ستون دین است."
models=[lm2,lm3,lm5]
model_names=["2-gram","3-gram","5-gram"]
for i in range(3):
    print("The perplexity of {} for the {} model is equal to:".format(sent2,model_names[i])+ str(models[i].perplexity(sent2)))

The perplexity of نماز ستون دین است. for the 2-gram model is equal to:21.512874985055344
The perplexity of نماز ستون دین است. for the 3-gram model is equal to:35.11615968792009
The perplexity of نماز ستون دین است. for the 5-gram model is equal to:61.68611798533915
